# Named Entity Recognition (NER) Assignment

In this assignment, you will fine-tune a pre-trained transformer model for a Named Entity Recognition (NER) task using the CoNLL-2003 dataset.

## Step 1: Setup
Let's start by installing the necessary libraries.

In [2]:
# Install the transformers library
!pip install transformers datasets
!pip install transformers[torch]
!pip install seqeval


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Install the necessary libraries
!pip install transformers datasets seqeval


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric, ClassLabel
import numpy as np

d:\Fusemachine_AI_Assignments\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Step 2: Load Dataset
We will use the CoNLL-2003 dataset, which is available through the `datasets` library.

In [6]:
# Load the CoNLL-2003 dataset
datasets = load_dataset("conll2003", trust_remote_code=True)
label_list = datasets["train"].features["ner_tags"].feature.names

# Display dataset structure
datasets

Generating test split: 100%|██████████| 3453/3453 [00:01<00:00, 2571.51 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

## Step 3: Tokenization
We need to tokenize the dataset using a pre-trained tokenizer.

In [7]:
# Load a pre-trained tokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize the dataset
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length")
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])# append label of patricular word index in label_ids # begin code() # end code()
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

d:\Fusemachine_AI_Assignments\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ramme\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
# Apply the tokenization and alignment function to the dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets.set_format("torch")

Map: 100%|██████████| 3453/3453 [00:02<00:00, 1170.12 examples/s]


In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

## Step 4: Load Pre-trained Model
We will use a pre-trained `DistilBERT` model for token classification.

In [10]:
# Load a pre-trained model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 5: Training
We will train the model using the `Trainer` API.

In [11]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the metric
metric = load_metric("seqeval", trust_remote_code=True)

d:\Fusemachine_AI_Assignments\venv\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ramme\AppData\Local\Temp\ipykernel_15096\666675623.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval", trust_remote_code=True)


In [12]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2) # get the argmax of predictions # begin code() # end code()

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    #create list of actual labels
    true_labels = [ # create list of actual labels # begin code() # end code()
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

  2%|▏         | 63/2634 [1:22:38<347:54:54, 487.16s/it]

## Step 6: Evaluation
We will evaluate the model on the test set.

In [ ]:
### Ex-5-Task-1
f1_score, precision, recall = None, None, None

### BEGIN SOLUTION
f1_score = 0.938055
precision = 0.933211
recall = 0.942949
# Replace the resulted metric obtained from training
# Put the metric resulted in the last epoch of the training
# raise NotImplementedError()
### END SOLUTION

In [ ]:
# Evaluate the model
results = trainer.evaluate(tokenized_datasets["test"])


# Print the evaluation results
print(results)

{'eval_loss': 0.11597243696451187, 'eval_precision': 0.8849496002780675, 'eval_recall': 0.9015580736543909, 'eval_f1': 0.8931766356779512, 'eval_accuracy': 0.978421449337784, 'eval_runtime': 65.7093, 'eval_samples_per_second': 52.55, 'eval_steps_per_second': 3.287, 'epoch': 3.0}


In [ ]:
# Function for NER inference
def ner_inference(texts):
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(device) # tokenize the texts and load it in cuda # begin code() # end code()
    with torch.no_grad():
        outputs = model(**inputs) # pass input to model # begin code() # end code()
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    results = []
    for i, text in enumerate(texts):
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][i]) # convert ids to token # begin code() # end code()
        pred = predictions[i].tolist()
        result = [(token, label_list[p]) for token, p in zip(tokens, pred) if token not in ["[CLS]", "[SEP]", "[PAD]"]]
        results.append(result)
    return results

In [ ]:
# Example sentences for NER inference
texts = ["Hugging Face Inc. is a company based in New York City.", "The quick brown fox jumps over the lazy dog."]
ner_results = ner_inference(texts)

# Print inference results
for i, result in enumerate(ner_results):
    print(f"Sentence {i+1}:")
    for token, label in result:
        print(f"{token}: {label}")

Sentence 1:
hugging: B-ORG
face: I-ORG
inc: I-ORG
.: O
is: O
a: O
company: O
based: O
in: O
new: B-LOC
york: I-LOC
city: I-LOC
.: O
Sentence 2:
the: O
quick: O
brown: O
fox: O
jumps: O
over: O
the: O
lazy: O
dog: O
.: O
